In [1]:
import os
import json
from os import listdir
from os.path import isfile, join
import h5py
import numpy as np
import itertools
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation

In [6]:
NUM_SAMPLE_FROM_POSTERIOR = 20
SMOOTHING_WINDOW_SIZE = 3
FPS = 100
C2F_LEVEL = 2

def compute_angular_velocity(q1, q2, dt):
    # Convert quaternions to scipy Rotation objects
    rot1 = Rotation.from_quat(q1)
    rot2 = Rotation.from_quat(q2)

    # Compute the relative rotation
    relative_rotation = rot2 * rot1.inv()

    # Convert the relative rotation to angle-axis representation
    angle = relative_rotation.magnitude()  # Rotation angle in radians
    axis = (
        relative_rotation.as_rotvec() / angle if angle != 0 else np.zeros(3)
    )  # Rotation axis

    # Compute angular velocity
    angular_velocity = (axis * angle) / dt
    return {
        "x": angular_velocity[0].astype(float).item(),
        "y": angular_velocity[1].astype(float).item(),
        "z": angular_velocity[2].astype(float).item(),
    }

def sample_from_posterior(log_probs_categories, option="rank"):
    log_probs = [item[0] for item in log_probs_categories]
    categories = [item[1] for item in log_probs_categories]
    num_categories = len(log_probs)

    if option == "uniform":
        def draw_single_sample():
            index = np.random.choice(num_categories)
            return categories[index]
    elif option == "veridical":
        def draw_single_sample():
            # see this: https://stackoverflow.com/questions/58339083/how-to-sample-from-a-log-probability-distribution
            gumbels = np.random.gumbel(size=num_categories)
            index = np.argmax(log_probs + gumbels)
            return categories[index]
    elif option == "rank":
        def draw_single_sample():
            weights = np.array([1 / (n + 1) for n in range(num_categories)])
            weights_norm = weights / weights.sum()
            index = np.random.choice(num_categories, p=weights_norm)
            return categories[index]
    elif option == "mix":
        def draw_single_sample():
            t = 0.5
            t * np.array(log_probs) + (1 - t) * (1 / num_categories)
            return
    else:
        raise NotImplementedError

    samples = []
    np.random.seed(42)
    for _ in range(NUM_SAMPLE_FROM_POSTERIOR):
        sample = draw_single_sample()
        samples.append(sample)
    return samples

def get_last_appearance(posterior_across_frames, o_id, start, stop, step):
    for index in range(start, stop, step):
        if o_id in list(posterior_across_frames["pose"][index].keys()):
            return index
    return None

def get_posterior_poses_for_frame_for_object(posterior_across_frames, o_id, frame, c2f_level=0):
    for idx, poses in posterior_across_frames["pose"][frame].items():
        if str(o_id) == idx:
            # print("yes!!")
            # poses = posterior_across_frames["pose"][frame][o_id]
            best_pose = poses[-1]
            pose_samples_from_posterior = [
                    [pose for pose in sample_from_posterior(poses[0][c2f_level])],
                    best_pose,
                ]
            return pose_samples_from_posterior

def compute_linear_velocity(
    pos_now,
    pos_last,
    dt,
):
    linear_vel = (pos_now - pos_last) / dt
    return {"x": linear_vel[0], "y": linear_vel[1], "z": linear_vel[2]}

def set_axes_equal(ax):
    """
    Make axes of 3D plot have equal scale so that spheres appear as spheres,
    cubes as cubes, etc.

    Input
      ax: a matplotlib axis, e.g., as output from plt.gca().
    """

    x_limits = ax.get_xlim3d()
    y_limits = ax.get_ylim3d()
    z_limits = ax.get_zlim3d()

    x_range = abs(x_limits[1] - x_limits[0])
    x_middle = np.mean(x_limits)
    y_range = abs(y_limits[1] - y_limits[0])
    y_middle = np.mean(y_limits)
    z_range = abs(z_limits[1] - z_limits[0])
    z_middle = np.mean(z_limits)

    # The plot bounding box is a sphere in the sense of the infinity
    # norm, hence I call half the max range the plot radius.
    plot_radius = 0.5*max([x_range, y_range, z_range])

    ax.set_xlim3d([x_middle - plot_radius, x_middle + plot_radius])
    ax.set_ylim3d([y_middle - plot_radius, y_middle + plot_radius])
    ax.set_zlim3d([z_middle - plot_radius, z_middle + plot_radius])

def mkdir(path):
    if not os.path.exists(path):
        os.mkdir(path)
    return path

def find_missing_values(nums):
    full_range = set(range(min(nums), max(nums) + 1))
    missing_values = sorted(list(full_range - set(nums)))
    return missing_values

In [13]:
scenarios = ['contain']
for scenario in scenarios:
    mkdir(f'/home/haoliangwang/data/b3d_tracking_results/new/{scenario}')
    scenario_path = f'/home/haoliangwang/data/physion_hdf5/{scenario}_all_movies/'
    onlyhdf5 = [
            f
            for f in listdir(scenario_path)
            if isfile(join(scenario_path, f)) and join(scenario_path, f).endswith(".hdf5")
        ]
    
    json_path = f'/home/haoliangwang/data/b3d_tracking_results/test/{scenario}'
    onlyjson = [
            f
            for f in listdir(json_path)
            if isfile(join(json_path, f)) and join(json_path, f).endswith(".json")
        ]
    for file in onlyjson:
        with h5py.File(os.path.join(scenario_path, file.replace(".json", ".hdf5")), "r") as f:
            object_ids = np.array(f["static"]["object_ids"])
            distractors = (
                np.array(f["static"]["distractors"])
                if np.array(f["static"]["distractors"]).size != 0
                else []
            )
            occluders = (
                np.array(f["static"]["occluders"])
                if np.array(f["static"]["occluders"]).size != 0
                else []
            )
            distractors_occluders = np.concatenate([distractors, occluders])
            if len(distractors_occluders):
                object_ids = object_ids[: -len(distractors_occluders)]
        missing = find_missing_values(object_ids)
        
        with open(os.path.join(json_path+'_optim', file)) as f:
            tracking_final_frame = json.load(f)
       
        new_tracking_final_frame = {}
        for k, v in tracking_final_frame.items():
            for miss in missing:
                if str(miss) not in v.keys():
                    print(miss, file)
                    v[str(miss)] = v['1']
            new_tracking_final_frame[k] = v

        with open(f'/home/haoliangwang/data/b3d_tracking_results/new/{scenario}/{file}', "w") as f:
            json.dump(new_tracking_final_frame, f)
            

2 pilot-containment-torus_0000.json
2 pilot-containment-torus_0000.json
2 pilot-containment-torus_0000.json
2 pilot-containment-torus_0000.json
2 pilot-containment-torus_0000.json
2 pilot-containment-torus_0000.json
2 pilot-containment-vase_0032.json
5 pilot-containment-vase_0032.json
2 pilot-containment-vase_0032.json
5 pilot-containment-vase_0032.json
2 pilot-containment-vase_0032.json
5 pilot-containment-vase_0032.json
2 pilot-containment-vase_0032.json
5 pilot-containment-vase_0032.json
2 pilot-containment-vase_0032.json
5 pilot-containment-vase_0032.json
2 pilot-containment-vase_0032.json
5 pilot-containment-vase_0032.json
2 pilot-containment-bowl_0015.json
5 pilot-containment-bowl_0015.json
2 pilot-containment-bowl_0015.json
5 pilot-containment-bowl_0015.json
2 pilot-containment-torus_0040.json
2 pilot-containment-torus_0040.json
2 pilot-containment-multi-bowl_0015.json
2 pilot-containment-multi-bowl_0015.json
2 pilot-containment-vase_0000.json
5 pilot-containment-vase_0000.json


In [3]:
# scenarios = ['support']
# for scenario in scenarios:
#     mkdir(f'/home/haoliangwang/data/b3d_tracking_results/new/{scenario}')
#     scenario_path = f'/home/haoliangwang/data/b3d_tracking_results/test/{scenario}'
#     onlyhdf5 = [
#             f
#             for f in listdir(scenario_path)
#             if isfile(join(scenario_path, f)) and join(scenario_path, f).endswith(".json")
#         ]
#     for file in onlyhdf5:
#         with open(os.path.join(scenario_path+'_optim', file)) as f:
#             tracking_final_frame = json.load(f)
#         with open(os.path.join(scenario_path+'_verbose', file)) as f:
#             tracking_per_frame = json.load(f)

#         new_per_frame_info = {'pose': []}
#         for frame in tracking_per_frame["pose"]:
#             new_frame_info = {}
#             for idx, poses in frame.items():
#                 best_pose = poses[-1]
#                 new_best_pose = list(itertools.chain.from_iterable(best_pose))
#                 new_samples = []
#                 for c2f_level in poses[0]:
#                     new_c2f_level = []
#                     for pose in c2f_level:
#                         new_c2f_level.append([pose[0], list(itertools.chain.from_iterable(pose[1:]))])
#                     new_samples.append(new_c2f_level)
#                 new_frame_info[idx] = [new_samples, new_best_pose]
#             new_per_frame_info['pose'].append(new_frame_info)
#         try:
#             for o_id in tracking_final_frame['model'].keys():
#                 get_posterior_poses_for_frame_for_object(new_per_frame_info, o_id, get_last_appearance(new_per_frame_info, o_id, len(new_per_frame_info["pose"]) - 1, -1, -1), c2f_level=C2F_LEVEL)
#         except:
#             print(f"{o_id}, {file}")        
#             new_tracking_final_frame = {}
#             for k, v in tracking_final_frame.items():
#                 v.pop('2', None)
#                 new_tracking_final_frame[k] = v

#             with open(f'/home/haoliangwang/data/b3d_tracking_results/new/{scenario}/{file}', "w") as f:
#                 json.dump(new_tracking_final_frame, f)
                

In [4]:
scenarios = ['collide', 'roll', 'support', 'dominoes', 'drop']
# scenarios = ['dominoes']
for scenario in scenarios:
    mkdir(f'/home/haoliangwang/data/b3d_tracking_results/new/{scenario}')
    scenario_path = f'/home/haoliangwang/data/b3d_tracking_results/test/{scenario}'
    onlyhdf5 = [
            f
            for f in listdir(scenario_path)
            if isfile(join(scenario_path, f)) and join(scenario_path, f).endswith(".json")
        ]
    for file in onlyhdf5:
        print(file)
        with open(os.path.join(scenario_path, file)) as f:
            tracking_final_frame = json.load(f)
        with open(os.path.join(scenario_path+'_verbose', file)) as f:
            tracking_per_frame = json.load(f)

        new_per_frame_info = {'pose': []}
        for frame in tracking_per_frame["pose"]:
            new_frame_info = {}
            for idx, poses in frame.items():
                best_pose = poses[-1]
                new_best_pose = list(itertools.chain.from_iterable(best_pose))
                new_samples = []
                for c2f_level in poses[0]:
                    new_c2f_level = []
                    for pose in c2f_level:
                        new_c2f_level.append([pose[0], list(itertools.chain.from_iterable(pose[1:]))])
                    new_samples.append(new_c2f_level)
                new_frame_info[idx] = [new_samples, new_best_pose]
            new_per_frame_info['pose'].append(new_frame_info)
        
        new_tracking_final_frame = {}
        for k, v in tracking_final_frame.items():
            if k == 'position':
                position_dict = dict(
                    [
                        (
                            str(o_id),
                            [
                                {
                                    "x": pose[0],
                                    "y": pose[1],
                                    "z": pose[2],
                                }
                                for pose in get_posterior_poses_for_frame_for_object(new_per_frame_info, o_id, get_last_appearance(new_per_frame_info, o_id, len(new_per_frame_info["pose"]) - 1, -1, -1), c2f_level=C2F_LEVEL)[0]
                            ],
                        )
                        for o_id in tracking_final_frame['model'].keys()
                    ]
                )
                new_tracking_final_frame[k] = position_dict
            elif k == 'rotation':
                rotation_dict = dict(
                    [
                        (
                            str(o_id),
                            [
                                {
                                    "x": pose[3],
                                    "y": pose[4],
                                    "z": pose[5],
                                    "w": pose[6],
                                }
                                for pose in get_posterior_poses_for_frame_for_object(new_per_frame_info, o_id, get_last_appearance(new_per_frame_info, o_id, len(new_per_frame_info["pose"]) - 1, -1, -1), c2f_level=C2F_LEVEL)[0]
                            ],
                        )
                        for o_id in tracking_final_frame['model'].keys()
                    ]
                )
                new_tracking_final_frame[k] = rotation_dict
            elif k == 'velocity':
                linear_velocity_dict = {}
                for o_id in v.keys():
                    if o_id not in list(new_per_frame_info["pose"][-1].keys()):
                        linear_velocity_dict[str(o_id)] = [{"x": 0.0, "y": 0.0, "z": 0.0} for _ in range(NUM_SAMPLE_FROM_POSTERIOR)]
                    else:
                        start_smooth_frame = get_last_appearance(new_per_frame_info, o_id, len(new_per_frame_info["pose"])-(SMOOTHING_WINDOW_SIZE + 1), len(new_per_frame_info["pose"]), 1)
                        sample_pt = get_posterior_poses_for_frame_for_object(new_per_frame_info, o_id, -1, c2f_level=C2F_LEVEL)
                        if start_smooth_frame == len(new_per_frame_info["pose"])-1:
                            anchor_pt = get_posterior_poses_for_frame_for_object(new_per_frame_info, o_id, get_last_appearance(new_per_frame_info, o_id, len(new_per_frame_info["pose"])-(SMOOTHING_WINDOW_SIZE + 2), -1, -1), c2f_level=C2F_LEVEL)
                            linear_velocity_dict[int(o_id)] = [
                                compute_linear_velocity(
                                    np.array(sample_pt[0][i][7:]),
                                    np.array(anchor_pt[-1][7:]),
                                    1 / FPS,
                                )
                                for i in range(NUM_SAMPLE_FROM_POSTERIOR)
                            ]
                        else:
                            anchor_pt = get_posterior_poses_for_frame_for_object(new_per_frame_info, o_id, start_smooth_frame, c2f_level=C2F_LEVEL)
                            linear_velocity_dict[int(o_id)] = [
                                compute_linear_velocity(
                                    np.array(sample_pt[0][i][7:]),
                                    np.array(anchor_pt[-1][7:]),
                                    (len(new_per_frame_info["pose"])-start_smooth_frame-1) / FPS,
                                )
                                for i in range(NUM_SAMPLE_FROM_POSTERIOR)
                            ]

                        # anchor_pt = get_posterior_poses_for_frame_for_object(new_per_frame_info, o_id, -SMOOTHING_WINDOW_SIZE-1, c2f_level=C2F_LEVEL)
                        # sample_pt = get_posterior_poses_for_frame_for_object(new_per_frame_info, o_id, -1, c2f_level=C2F_LEVEL)
                        # linear_velocity_dict[str(o_id)] = [
                        #     compute_linear_velocity(
                        #         np.array(sample_pt[0][i][7:]),
                        #         np.array(anchor_pt[-1][7:]),  # using optim pose for window frame
                        #         SMOOTHING_WINDOW_SIZE / FPS,
                        #     )
                        #     for i in range(NUM_SAMPLE_FROM_POSTERIOR)
                        # ]
                new_tracking_final_frame[k] = linear_velocity_dict
            elif k == 'angular_velocity':
                angular_velocity_dict = {}
                for o_id in v.keys():
                    if o_id not in list(new_per_frame_info["pose"][-1].keys()):
                        angular_velocity_dict[str(o_id)] = [{"x": 0.0, "y": 0.0, "z": 0.0} for _ in range(NUM_SAMPLE_FROM_POSTERIOR)]
                    else:
                        start_smooth_frame = get_last_appearance(new_per_frame_info, o_id, len(new_per_frame_info["pose"])-(SMOOTHING_WINDOW_SIZE + 1), len(new_per_frame_info["pose"]), 1)
                        sample_pt = get_posterior_poses_for_frame_for_object(new_per_frame_info, o_id, -1, c2f_level=C2F_LEVEL)
                        if start_smooth_frame == len(new_per_frame_info["pose"])-1:
                            anchor_pt = get_posterior_poses_for_frame_for_object(new_per_frame_info, o_id, get_last_appearance(new_per_frame_info, o_id, len(new_per_frame_info["pose"])-(SMOOTHING_WINDOW_SIZE + 2), -1, -1), c2f_level=C2F_LEVEL)
                            angular_velocity_dict[int(o_id)] = [
                                compute_angular_velocity(
                                    np.array(anchor_pt[-1][3:7]),
                                    np.array(sample_pt[0][i][3:7]),
                                    1 / FPS,
                                )
                                for i in range(NUM_SAMPLE_FROM_POSTERIOR)
                            ]
                        else:
                            anchor_pt = get_posterior_poses_for_frame_for_object(new_per_frame_info, o_id, start_smooth_frame, c2f_level=C2F_LEVEL)
                            angular_velocity_dict[int(o_id)] = [
                                compute_angular_velocity(
                                    np.array(anchor_pt[-1][3:7]),
                                    np.array(sample_pt[0][i][3:7]),
                                    (len(new_per_frame_info["pose"])-start_smooth_frame-1) / FPS,
                                )
                                for i in range(NUM_SAMPLE_FROM_POSTERIOR)
                            ]

                        # anchor_pt = get_posterior_poses_for_frame_for_object(new_per_frame_info, o_id, -SMOOTHING_WINDOW_SIZE-1, c2f_level=C2F_LEVEL)
                        # sample_pt = get_posterior_poses_for_frame_for_object(new_per_frame_info, o_id, -1, c2f_level=C2F_LEVEL)
                        # angular_velocity_dict[str(o_id)] = [
                        #     compute_angular_velocity(
                        #         np.array(anchor_pt[-1][3:7]),
                        #         np.array(sample_pt[0][i][3:7]),
                        #         SMOOTHING_WINDOW_SIZE / FPS,
                        #     )
                        #     for i in range(NUM_SAMPLE_FROM_POSTERIOR)
                        # ]
                new_tracking_final_frame[k] = angular_velocity_dict
            else:
                new_tracking_final_frame[k] = v

        with open(f'/home/haoliangwang/data/b3d_tracking_results/new/{scenario}/{file}', "w") as f:
            json.dump(new_tracking_final_frame, f)

pilot_it2_collision_non-sphere_box_0030.json
pilot_it2_collision_tiny_ball_box_0037.json
pilot_it2_collision_non-sphere_tdw_1_dis_1_occ_0023.json
pilot_it2_collision_assorted_targets_box_0011.json
pilot_it2_collision_non-sphere_box_0036.json
pilot_it2_collision_yeet_tdw_1_dis_1_occ_0036.json
pilot_it2_collision_non-sphere_tdw_1_dis_1_occ_0000.json
pilot_it2_collision_yeet_box_0007.json
pilot_it2_collision_non-sphere_tdw_1_dis_1_occ_0017.json
pilot_it2_collision_non-sphere_box_0021.json
pilot_it2_collision_yeet_tdw_1_dis_1_occ_0021.json
pilot_it2_collision_non-sphere_box_0015.json
pilot_it2_collision_tiny_ball_box_0024.json
pilot_it2_collision_yeet_box_0030.json
pilot_it2_collision_non-sphere_tdw_1_dis_1_occ_0024.json
pilot_it2_collision_assorted_targets_box_0007.json
pilot_it2_collision_simple_box_0006.json
pilot_it2_collision_non-sphere_box_0035.json
pilot_it2_collision_non-sphere_box_0029.json
pilot_it2_collision_simple_box_1_dis_1_occ_0014.json
pilot_it2_collision_tiny_ball_box_0029

In [4]:
config = 'pilot_dominoes_0mid_d3chairs_o1plants_tdwroom'
trial_index = str(10)

with open(f'/home/haoliangwang/data/b3d_tracking_results/test/dominoes_verbose/{config}_{trial_index.zfill(4)}.json') as f:
    tracking_per_frame = json.load(f)
with open(f'/home/haoliangwang/data/b3d_tracking_results/test/dominoes/{config}_{trial_index.zfill(4)}.json') as f:
    tracking_final_frame = json.load(f)

In [5]:
new_per_frame_info = {'pose': []}
for frame in tracking_per_frame["pose"]:
    new_frame_info = {}
    for idx, poses in frame.items():
        best_pose = poses[-1]
        new_best_pose = list(itertools.chain.from_iterable(best_pose))
        new_samples = []
        for c2f_level in poses[0]:
            new_c2f_level = []
            for pose in c2f_level:
                new_c2f_level.append([pose[0], list(itertools.chain.from_iterable(pose[1:]))])
            new_samples.append(new_c2f_level)
        new_frame_info[idx] = [new_samples, new_best_pose]
    new_per_frame_info['pose'].append(new_frame_info)

In [ ]:
new_tracking_final_frame = {}
info_dict = dict(
    [
        (
            str(o_id),
            [
                {
                    "px": pose[0],
                    "py": pose[1],
                    "pz": pose[2],
                    "rx": pose[3],
                    "ry": pose[4],
                    "rz": pose[5],
                    "rw": pose[6],
                }
                for pose in get_posterior_poses_for_frame_for_object(new_per_frame_info, o_id, get_last_appearance(new_per_frame_info, o_id, len(new_per_frame_info["pose"]) - 1, -1, -1), c2f_level=C2F_LEVEL)[0]
            ],
        )
        for o_id in tracking_final_frame['model'].keys()
    ]
)
for k, v in tracking_final_frame.items():
    if k == 'position':
        position_dict = {}
        for object_id, samples in info_dict.items():
            position_dict[object_id] = []
            for sample in samples:
                position_dict[object_id].append({'x': sample['px'], 'y': sample['py'], 'z': sample['pz']})
        new_tracking_final_frame[k] = position_dict
    elif k == 'rotation':
        rotation_dict = {}
        for object_id, samples in info_dict.items():
            rotation_dict[object_id] = []
            for sample in samples:
                rotation_dict[object_id].append({'x': sample['rx'], 'y': sample['ry'], 'z': sample['rz'], 'w': sample['rw']})
        new_tracking_final_frame[k] = rotation_dict
    elif k == 'velocity':
        linear_velocity_dict = {}
        for o_id in v.keys():
            if o_id not in list(new_per_frame_info["pose"][-1].keys()):
                linear_velocity_dict[str(o_id)] = [{"x": 0.0, "y": 0.0, "z": 0.0} for _ in range(NUM_SAMPLE_FROM_POSTERIOR)]
            else:
                anchor_pt = get_posterior_poses_for_frame_for_object(new_per_frame_info, o_id, -SMOOTHING_WINDOW_SIZE-1, c2f_level=C2F_LEVEL)
                sample_pt = get_posterior_poses_for_frame_for_object(new_per_frame_info, o_id, -1, c2f_level=C2F_LEVEL)
                linear_velocity_dict[str(o_id)] = [
                    compute_linear_velocity(
                        np.array(sample_pt[0][i][7:]),
                        np.array(anchor_pt[-1][7:]),  # using optim pose for window frame
                        SMOOTHING_WINDOW_SIZE / FPS,
                    )
                    for i in range(NUM_SAMPLE_FROM_POSTERIOR)
                ]
        new_tracking_final_frame[k] = linear_velocity_dict
    elif k == 'angular_velocity':
        angular_velocity_dict = {}
        for o_id in v.keys():
            if o_id not in list(new_per_frame_info["pose"][-1].keys()):
                angular_velocity_dict[str(o_id)] = [{"x": 0.0, "y": 0.0, "z": 0.0} for _ in range(NUM_SAMPLE_FROM_POSTERIOR)]
            else:
                anchor_pt = get_posterior_poses_for_frame_for_object(new_per_frame_info, o_id, -SMOOTHING_WINDOW_SIZE-1, c2f_level=C2F_LEVEL)
                sample_pt = get_posterior_poses_for_frame_for_object(new_per_frame_info, o_id, -1, c2f_level=C2F_LEVEL)
                angular_velocity_dict[str(o_id)] = [
                    compute_angular_velocity(
                        np.array(anchor_pt[-1][3:7]),
                        np.array(sample_pt[0][i][3:7]),
                        SMOOTHING_WINDOW_SIZE / FPS,
                    )
                    for i in range(NUM_SAMPLE_FROM_POSTERIOR)
                ]
        new_tracking_final_frame[k] = angular_velocity_dict
    else:
        new_tracking_final_frame[k] = v

0
32
18
25
25
44
44
44
None


TypeError: list indices must be integers or slices, not NoneType

In [7]:
with open("/home/haoliangwang/data/b3d_tracking_results/test.json", "w") as f:
    json.dump(new_tracking_final_frame, f)